In [46]:
import networkx as nx
import random
from ortools.linear_solver import pywraplp

# Configurações gerais
n = 100  # número de vértices
p = 0.1  # probabilidade de aresta
random.seed(42)

# Gerando o grafo aleatório
G = nx.gnp_random_graph(n, p)
for (u, v) in G.edges():
    G.edges[u, v]['weight'] = random.randint(10, 500)

# Visualização básica (opcional)
print("Número de vértices:", G.number_of_nodes())
print("Número de arestas:", G.number_of_edges())


Número de vértices: 100
Número de arestas: 474


In [47]:
import itertools

def solve_with_subtour_elimination(G, d):
    # Criando o solver
    solver = pywraplp.Solver.CreateSolver('SCIP')
    if not solver:
        return None

    # Variáveis de decisão: x[u, v] = 1 se a aresta (u, v) é selecionada, 0 caso contrário
    x = {}
    for u, v in G.edges():
        x[u, v] = solver.BoolVar(f'x_{u}_{v}')
        x[v, u] = solver.BoolVar(f'x_{v}_{u}')  # Considerando grafo não direcionado

    # Função objetivo: minimizar o custo das arestas selecionadas
    solver.Minimize(solver.Sum(G.edges[u, v]['weight'] * x[u, v] for u, v in G.edges()))

    # 1. Restrição de árvore geradora no subgrafo dos centrais: 
    centrais = d.keys()
    solver.Add(solver.Sum(x[u, v] for u, v in G.edges() if u in centrais and v in centrais) == len(centrais) - 1)

    # 2. Eliminação de ciclos (Subtour Elimination Constraints - SECs) no subgrafo dos centrais
    for subset in itertools.chain.from_iterable(itertools.combinations(centrais, r) for r in range(2, len(centrais))):
        solver.Add(solver.Sum(x[u, v] for u, v in itertools.combinations(subset, 2) if G.has_edge(u, v)) <= len(subset) - 1)

    # 3. Restrição de grau mínimo para os vértices centrais
    for v in centrais:
        solver.Add(solver.Sum(x[u, v] for u in G.neighbors(v)) >= d[v])

    # 4. Cada terminal deve ser conectado a apenas um vértice central (grau 1)
    terminals = set(G.nodes()) - set(centrais)  # Supondo que vértices não em 'd' são terminais
    for t in terminals:
        solver.Add(solver.Sum(x[t, v] for v in G.neighbors(t)) == 1)

    # 5. Garantir que as variáveis são binárias (0 ou 1)
    for u, v in G.edges():
        solver.Add(x[u, v] == x[v, u])  # Impor que seja binário

    # Limite de tempo de execução (opcional)
    solver.SetTimeLimit(1800 * 1000)  # 1800 segundos

    # Resolução do problema
    status = solver.Solve()
    if status == pywraplp.Solver.OPTIMAL:
        print('Solução ótima encontrada!')
        return solver.Objective().Value(), [(u, v) for u, v in G.edges() if x[u, v].solution_value() > 0.5]
    else:
        print('Nenhuma solução ótima encontrada.')
        return None

In [48]:
def solve_with_mtz(G):
    solver = pywraplp.Solver.CreateSolver('SCIP')
    if not solver:
        return None

    n = G.number_of_nodes()
    c = n  # um número grande o suficiente

    # Variáveis de decisão
    y = {}
    u_labels = {}  # Alterando o nome para evitar colisão com a variável 'u'
    
    for node in G.nodes():
        u_labels[node] = solver.IntVar(0, c - 1, f'u_{node}')
        for neighbor in G.nodes():
            y[node, neighbor] = solver.BoolVar(f'y_{node}_{neighbor}')

    # Função objetivo
    solver.Minimize(solver.Sum(G.edges[u, v]['weight'] * y[u, v] for u, v in G.edges()))

    # Restrição de árvore geradora direcionada
    for v in G.nodes():
        if v != 0:  # assumindo que o nó 0 é a raiz
            solver.Add(solver.Sum(y[u, v] for u, v in G.edges() if u == v or v == v) == 1)

    # Rótulos nos vértices (MTZ)
    for u, v in G.edges():
        solver.Add(u_labels[u] - u_labels[v] + c * y[u, v] <= c - 1)

    # Limite de tempo de execução
    solver.SetTimeLimit(1800 * 1000)  # 1800 segundos

    # Resolução do problema
    status = solver.Solve()
    if status == pywraplp.Solver.OPTIMAL:
        print('Solução ótima encontrada!')
        return solver.Objective().Value()
    else:
        print('Nenhuma solução encontrada.')
        return None


In [49]:
import networkx as nx

def read_graph_from_file(file_path):
    with open(file_path, 'r') as f:
        # Primeira linha: n (vértices), nc (centrais), m (arestas)
        n, nc, m = map(int, f.readline().strip().split())
        
        # Próximas nc linhas: centrais e graus mínimos
        centrais = {}
        for _ in range(nc):
            i, d = map(int, f.readline().strip().split())
            centrais[i] = d
        
        # Próximas m linhas: arestas e custos
        G = nx.Graph()
        for _ in range(m):
            i, j, c = map(int, f.readline().strip().split())
            G.add_edge(i, j, weight=c)
        
    return G, centrais

# Exemplo de uso
file_path = './Instancias/tb8ch15_0.txt'
grafo, centrais = read_graph_from_file(file_path)

# Exibindo o grafo e as informações dos vértices centrais
print("Grafo:", grafo.edges(data=True))
print("Centrais e graus mínimos:", centrais)


Grafo: [(1, 2, {'weight': 396}), (1, 3, {'weight': 283}), (1, 4, {'weight': 198}), (1, 5, {'weight': 256}), (1, 6, {'weight': 268}), (1, 7, {'weight': 260}), (1, 8, {'weight': 290}), (1, 9, {'weight': 228}), (1, 10, {'weight': 109}), (1, 11, {'weight': 257}), (1, 12, {'weight': 168}), (1, 13, {'weight': 170}), (1, 14, {'weight': 271}), (1, 15, {'weight': 343}), (1, 16, {'weight': 113}), (1, 17, {'weight': 37}), (1, 18, {'weight': 233}), (1, 19, {'weight': 299}), (1, 20, {'weight': 324}), (1, 21, {'weight': 351}), (2, 3, {'weight': 676}), (2, 4, {'weight': 535}), (2, 5, {'weight': 355}), (2, 6, {'weight': 405}), (2, 7, {'weight': 650}), (2, 8, {'weight': 266}), (2, 9, {'weight': 306}), (2, 10, {'weight': 320}), (2, 11, {'weight': 389}), (2, 12, {'weight': 485}), (2, 13, {'weight': 477}), (2, 14, {'weight': 667}), (2, 15, {'weight': 638}), (2, 16, {'weight': 506}), (2, 17, {'weight': 361}), (2, 18, {'weight': 519}), (2, 19, {'weight': 683}), (2, 20, {'weight': 675}), (2, 21, {'weight': 7

In [50]:
import time

def compare_formulations(G,d):
    # Suponha que 'd' é um dicionário que você já definiu contendo os graus mínimos dos vértices centrais

    start_time = time.time()
    obj_value_subtour = solve_with_subtour_elimination(G, d)
    time_subtour = time.time() - start_time

    print(f"Subtour Elimination: Objective Value = {obj_value_subtour}, Time = {time_subtour} seconds")

    # start_time = time.time()
    # obj_value_mtz = solve_with_mtz(G, d)  # Supondo que você tenha uma função equivalente para MTZ
    # time_mtz = time.time() - start_time

    # print(f"MTZ: Objective Value = {obj_value_mtz}, Time = {time_mtz} seconds")

# Exemplo de comparação
compare_formulations(grafo, centrais)


Solução ótima encontrada!
Subtour Elimination: Objective Value = (2972.0, [(1, 5), (1, 7), (1, 16), (1, 17), (2, 6), (2, 8), (2, 10), (3, 7), (3, 14), (3, 21), (4, 5), (4, 13), (4, 18), (5, 8), (6, 11), (6, 12), (7, 15), (7, 19), (7, 20), (8, 9)]), Time = 0.03656148910522461 seconds
